In [1]:
import os, warnings
import matplotlib.pyplot as plt
from matplotlib import gridspec

import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing import image_dataset_from_directory

ds_train = image_dataset_from_directory(
    'input/car-or-truck/train',
    labels = 'inferred',
    label_mode = 'binary',
    image_size = [128, 128],
    interpolation = 'nearest',
    batch_size = 64,
    shuffle = True,
).cache()

ds_valid = image_dataset_from_directory(
    'input/car-or-truck/valid',
    labels = 'inferred',
    label_mode = 'binary',
    image_size = [128, 128],
    interpolation = 'nearest',
    batch_size = 64,
    shuffle = False,
).cache()

def convert_to_float(image, label):
    image = tf.image.convert_image_dtype(image, dtype=tf.float32)
    return image, label

AUTOTUNE = tf.data.experimental.AUTOTUNE
ds_train = (
    ds_train
    .map(convert_to_float)
    .cache()
    .prefetch(buffer_size=AUTOTUNE)
)
ds_valid = (
    ds_valid
    .map(convert_to_float)
    .cache()
    .prefetch(buffer_size=AUTOTUNE)
)


Found 5117 files belonging to 2 classes.
Found 5051 files belonging to 2 classes.


In [4]:
from tensorflow import keras
from tensorflow.keras import layers

from tensorflow.keras.layers.experimental import preprocessing

model = keras.Sequential([
    preprocessing.RandomFlip('horizontal'),
    preprocessing.RandomContrast(0.5),
    
    layers.Conv2D(filters=32, kernel_size=5, activation="relu", padding='same',
                  input_shape=[128, 128, 3]),
    layers.MaxPool2D(),

    layers.Conv2D(filters=64, kernel_size=3, activation="relu", padding='same'),
    layers.Conv2D(filters=64, kernel_size=3, activation="relu", padding='same'),
    layers.MaxPool2D(),

    layers.Conv2D(filters=128, kernel_size=3, activation="relu", padding='same'),
    layers.MaxPool2D(),

    layers.Flatten(),
    layers.Dense(units=6, activation="relu"),
    layers.Dense(units=1, activation="sigmoid"),
])

In [5]:
model.compile(
    optimizer = tf.keras.optimizers.Adam(epsilon = 0.01),
    loss = 'binary_crossentropy',
    metrics = ['binary_accuracy']
)

display(type(ds_train))

history = model.fit(
    ds_train,
    validation_data=ds_valid,
    epochs=40,
)

tensorflow.python.data.ops.dataset_ops.PrefetchDataset

Epoch 1/40
80/80 [==============================] - 145s 2s/step - loss: 0.6784 - binary_accuracy: 0.5730 - val_loss: 0.6700 - val_binary_accuracy: 0.5785
Epoch 2/40
80/80 [==============================] - 115s 1s/step - loss: 0.6594 - binary_accuracy: 0.6084 - val_loss: 0.6648 - val_binary_accuracy: 0.5811
Epoch 3/40
80/80 [==============================] - 126s 2s/step - loss: 0.6535 - binary_accuracy: 0.6152 - val_loss: 0.6697 - val_binary_accuracy: 0.5933
Epoch 4/40
80/80 [==============================] - 117s 1s/step - loss: 0.6479 - binary_accuracy: 0.6174 - val_loss: 0.6596 - val_binary_accuracy: 0.6025
Epoch 5/40
80/80 [==============================] - 116s 1s/step - loss: 0.6347 - binary_accuracy: 0.6398 - val_loss: 0.6717 - val_binary_accuracy: 0.5941
Epoch 6/40
80/80 [==============================] - 115s 1s/step - loss: 0.6213 - binary_accuracy: 0.6488 - val_loss: 0.6517 - val_binary_accuracy: 0.6173
Epoch 7/40
80/80 [==============================] - 115s 1s/step - los

In [ ]:
from IPython.core.display_functions import display
import pandas as pd

history_frame = pd.DataFrame(history.history)
display(history_frame)
history_frame[['loss']].plot()
history_frame[['binary_accuracy']].plot();

display(history_frame)